# Studio 1: Getting Started - Multi-Node Training with Monarch & Lightning

Welcome! This notebook will guide you through running **distributed multi-node training** using **Monarch** (Meta's distributed actor framework) with **TorchTitan** (PyTorch's large-scale LLM training library) on **Lightning AI** infrastructure.

<div align="center">
  <img src="./assets/NB_Monarch_Lightning.svg" alt="Monarch Lightning Architecture" width="800"/>
</div>

## What You'll Learn

By the end of this notebook, you'll:
- Set up TorchTitan, Monarch, and Lightning SDK
- Launch a multi-node training job on Lightning AI
- Run distributed Llama-3-8B training across multiple GPUs
- Monitor and manage your distributed training

## Prerequisites

- Lightning AI account with access to GPU machines (L40S recommended)
- Hugging Face account with Llama model access
- Basic understanding of distributed training concepts

## Next Steps

After completing this studio:
- **Studio 2**: Learn workspace synchronization for hot-reloading configs without restarting jobs
- **Studio 3**: Master interactive debugging across distributed nodes

Let's get started!

---

# Part I: Environment Setup

Before running distributed training, we need to install dependencies. Follow the steps below.

## Install TorchTitan

Clone the TorchTitan repository, install the nightly PyTorch build with CUDA 12.6 support, and install TorchTitan:

```bash
git clone https://github.com/pytorch/torchtitan.git
cd torchtitan
pip3 install --pre torch --index-url https://download.pytorch.org/whl/nightly/cu126 --force-reinstall
pip install .
```

## Download Llama-3-8B Model Assets

Download the Llama-3.1-8B tokenizer from Hugging Face. You'll need a Hugging Face token with access to the Llama models:

```bash
python scripts/download_hf_assets.py \
    --repo_id meta-llama/Llama-3.1-8B \
    --assets tokenizer \
    --hf_token=YOUR_HUGGINGFACE_TOKEN_KEY
```

Replace `YOUR_HUGGINGFACE_TOKEN_KEY` with your actual Hugging Face token.

## Install Monarch

Install Monarch from the GitHub repository following the Ubuntu installation instructions:

```bash
git clone https://github.com/meta-pytorch/monarch.git
cd monarch
# Follow the Ubuntu installation instructions from the repository
```

For detailed installation steps, visit: https://github.com/meta-pytorch/monarch

## Setup Weights & Biases

Install wandb for experiment tracking:

```bash
pip install wandb
wandb login
```

Follow the prompts to authenticate with your wandb account.

## Update the Lightning SDK

Install the latest version of Lightning SDK for IP sharing features:

```bash
pip install -U lightning_sdk
```

## Verify Installations

After completing the installation steps above, verify that TorchTitan and Monarch are properly installed:

In [ ]:
# Verify TorchTitan installation
import torchtitan
print("TorchTitan is installed successfully")

# Verify Monarch installation
import monarch
print("Monarch is installed successfully")

# Verify PyTorch and CUDA
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

---

# Part II: Multi-Node Training with Monarch and Lightning

Now that the environment is set up, we'll configure and launch distributed training across multiple nodes.

## Import Lightning SDK Components

Import the necessary classes from Lightning SDK to manage multi-machine training jobs.

In [ ]:
from lightning_sdk import Machine, MMT, Studio

## Configure Training Job Parameters

Set up the configuration for the multi-node training job. We'll start with **2 nodes** to keep things manageable.

> **Note:** You can easily scale this up to 16+ nodes once you're comfortable with the workflow!

In [ ]:
# Configuration
import os

NUM_NODES = 2
NUM_GPUS = 8
TEAMSPACE = "general"  # Replace with your teamspace
USER = "your-username"  # Replace with your username
MMT_JOB_NAME = f"Monarch-MMT-{NUM_NODES}-nodes"

# Remote allowed port range for worker nodes
REMOTE_ALLOWED_PORT_RANGE = "26601..26611"

# To force Monarch to use V0 for this Notebook (This will be removed in the future)
os.environ["MONARCH_V0_WORKAROUND_DO_NOT_USE"] = "1"
os.environ["MONARCH_FILE_LOG"] = "debug"

## Define MMT Job Launch Function

Create a function to launch a multi-machine training (MMT) job using Lightning SDK.

In [ ]:
def launch_mmt_job(num_nodes=2, teamspace="my-teamspace", user="my-user"):
    """
    Launch a multi-machine training job using Lightning SDK's MMT API.
    """

    studio = Studio()

    # Install the MMT plugin before running the actual job
    studio.install_plugin("multi-machine-training")

    print(f"Launching MMT job with {num_nodes} nodes...")

    # Machine with L40S GPUs
    machine_type = getattr(Machine, f"L40S_X_{NUM_GPUS}")

    job = MMT.run(
        command="process_allocator",
        name=MMT_JOB_NAME,
        machine=machine_type,
        studio=studio,
        num_machines=num_nodes,
        env={
            "CUDA_VISIBLE_DEVICES": "0,1,2,3,4,5,6,7",
            "MONARCH_FILE_LOG": "debug",
            "HYPERACTOR_REMOTE_ALLOC_ALLOWED_PORT_RANGE": REMOTE_ALLOWED_PORT_RANGE,
            "HYPERACTOR_REMOTE_ALLOC_BIND_TO_INADDR_ANY": "true",
            "WORKSPACE_DIR": "/tmp",
        },
    )

    print(f"Job started with ID: {job.name}")
    print(f"Job status: {job.status}")

    return job, studio

## Launch the Multi-Node Training Job

Execute the launch function to start the distributed training infrastructure.

In [ ]:
# Launch the job
job, studio = launch_mmt_job(
    num_nodes=NUM_NODES, teamspace=TEAMSPACE, user=USER
)

print(f"\nJob launched. You can monitor it using: job.status")
print(f"To stop the job: job.stop()")
print(f"To clean up: studio.stop()")

## Monitor Job Status

You can monitor your job through the MMT plugin in Lightning AI. The nodes will go through these stages:

1. **Pending** - Waiting for resources
2. **Setting up** - Installing dependencies and snapshotting environment
3. **Ready** - All nodes ready with `process_allocator` running

Wait for all nodes to show **Ready** status before proceeding to the next cell.

In [ ]:
# Check job status
print(f"Current job status: {job.status}")

## Set Up Process Mesh from Job

Initialize the Monarch process mesh using the launched Lightning job. This creates the distributed computing mesh that connects all nodes and GPUs.

> **Important:** Make sure the `process_allocator` process is running on all nodes before running this cell!

In [ ]:
from utils.mesh_utils import setup_proc_mesh_from_job

proc_mesh = setup_proc_mesh_from_job(job, NUM_NODES, NUM_GPUS)
print("\nProcess mesh initialized successfully!")

---

# Run TorchTitan Training for Llama-3-8B

Now we'll define a Monarch Actor that wraps TorchTitan's training functionality and run distributed training.

## Generate Job Name Helper

Create a unique job name for tracking.

In [ ]:
import getpass

def get_job_name(num_hosts: int, num_gpus_per_host: int):
    return f"monarch-{getpass.getuser()}-hosts{num_hosts}-gpus{num_gpus_per_host}"

print(get_job_name(num_hosts=NUM_NODES, num_gpus_per_host=NUM_GPUS))

## Define TorchTitan Trainer Actor

Create the `TitanTrainerWrapper` class, a Monarch Actor that wraps TorchTitan's training functionality.

In [ ]:
import os
import sys
import logging
from monarch.actor import ProcMesh, Actor, endpoint, current_rank
import socket
from torchtitan.tools.logging import init_logger, logger
from torchtitan.train import Trainer
from typing import Optional
import torch
from torchtitan.config import JobConfig


class TitanTrainerWrapper(Actor):
    def __init__(self, job_config: JobConfig):
        self.rank = current_rank().rank
        self.job_config = job_config

    def _rprint(self, msg):
        """Helper method to print with rank information."""
        print(f"{self.rank=} {msg}")

    @endpoint
    def init(self):
        logging.getLogger().addHandler(logging.StreamHandler(sys.stderr))
        print(f"Initializing actor: {self.rank} {current_rank()=} {socket.gethostname()=}")

    @endpoint
    def train(self):
        logger.info("Starting training")
        config = self.job_config
        trainer: Optional[Trainer] = None

        try:
            trainer = Trainer(config)
            trainer.train()

            if config.checkpoint.create_seed_checkpoint:
                assert (
                    int(os.environ["WORLD_SIZE"]) == 1
                ), "Must create seed checkpoint using a single device, to disable sharding."
                assert config.checkpoint.enable, "Must enable checkpointing when creating a seed checkpoint."
                trainer.checkpointer.save(curr_step=0)
                logger.info("Created seed checkpoint")
            else:
                trainer.train()
        finally:
            if trainer:
                trainer.close()

            if torch.distributed.is_initialized():
                torch.distributed.destroy_process_group()
                logger.info("Process group destroyed.")
        print("Done training")

## Define Async Main Training Function

Set up the main asynchronous function that orchestrates distributed training.

In [ ]:
from torchtitan.config import ConfigManager, JobConfig
from monarch.tools.network import AddrType
from monarch.utils import setup_env_for_distributed


async def async_main(job_config: JobConfig):
    torch.use_deterministic_algorithms(True)
    job_name = get_job_name(NUM_NODES, NUM_GPUS)

    # Use IPv4 for MASTER_ADDR
    await setup_env_for_distributed(proc_mesh, use_ipaddr=AddrType.IPv4)

    await proc_mesh.logging_option(stream_to_client=True, aggregate_window_sec=3)

    print(job_config)
    print(f"Spawning meshes on {job_name}")

    trainer_actor = proc_mesh.spawn("trainer_actor", TitanTrainerWrapper, job_config)

    await trainer_actor.init.call()
    await trainer_actor.train.call()

## Initialize Logger and Run Training

Configure the TorchTitan logger, set up training parameters, and execute the training pipeline.

> **Note:** This will train Llama-3-8B for 25 steps. Adjust the paths below to match your setup.

In [ ]:
init_logger()
config_manager = ConfigManager()

job_name = get_job_name(NUM_NODES, NUM_GPUS)

manual_args = [
    "--job.config_file",
    os.path.expanduser("/teamspace/studios/this_studio/torchtitan/torchtitan/models/llama3/train_configs/llama3_8b.toml"),
    "--model.tokenizer-path",
    "/teamspace/studios/this_studio/torchtitan/assets/hf/Llama-3.1-8B",
    "--training.steps",
    "25",
    "--training.dataset_path",
    "/teamspace/studios/this_studio/torchtitan/tests/assets/c4_test",
    "--job.dump_folder",
    "/teamspace/studios/this_studio/torchtitan/outputs/" + job_name,
    "--training.seq_len",
    "1024",
]
config = config_manager.parse_args(manual_args)
await async_main(config)

---

# 🎉 Congratulations! 🎉

You just ran **interactive distributed training** for a Llama-3-8B model in a Jupyter notebook using **Monarch actors** and **Lightning infrastructure**!

## What You Accomplished

- Launched a multi-node training job on Lightning AI
- Set up a distributed process mesh with Monarch
- Ran TorchTitan training across multiple GPUs and nodes
- Monitored training with aggregated logging

## Key Benefits

- **Flexibility**: Change configurations and relaunch training without restarting nodes
- **Observability**: Monarch aggregates logs from all ranks
- **Scalability**: Easily scale from 2 to 16+ nodes by changing `NUM_NODES`

## Next Steps

Now that you've mastered the basics, explore advanced Monarch features:

### 📚 Studio 2: Workspace Synchronization (Recommended Next)
Learn how to:
- Sync local code/config changes to remote nodes **without restarting**
- Hot-reload training configurations
- Iterate faster on distributed training

### 🐛 Studio 3: Interactive Debugging
Master advanced debugging:
- Set breakpoints in distributed actors
- Debug specific ranks interactively
- Inspect environment variables across nodes

---

## Cleanup

When you're done, remember to stop the process mesh and clean up resources:

In [ ]:
# Stop the process mesh
await proc_mesh.stop()

# Stop the Lightning job
job.stop()

print("Cleanup complete!")